In [ ]:
import numpy as np
from collections import deque

In [ ]:
PUZZLE_DIM = 3
MOVES = [(-1, 0, "U"), (1, 0, "D"), (0, -1, "L"), (0, 1, "R")]

In [ ]:
def generate_initial_state():
    """
    Generate a random solvable initial state for an n x n puzzle.

    Returns:
        tuple: A random, solvable initial state.
    """
    goal_state = tuple(range(1, PUZZLE_DIM * PUZZLE_DIM)) + (0,)
    while True:
        state = list(goal_state)
        np.random.shuffle(state)
        state = tuple(state)
        if is_solvable(state):
            return state

In [ ]:
def is_solvable(state):
    """
    Check if a given puzzle state is solvable.

    Args:
        state (tuple): The current puzzle state.

    Returns:
        bool: True if the state is solvable, False otherwise.
    """
    # Count inversions
    inversion_count = 0
    state_list = [tile for tile in state if tile != 0]  # Exclude the blank tile
    for i in range(len(state_list)):
        for j in range(i + 1, len(state_list)):
            if state_list[i] > state_list[j]:
                inversion_count += 1

    if PUZZLE_DIM % 2 == 1:  # Odd grid
        return inversion_count % 2 == 0
    else:  # Even grid
        # Find the row of the blank tile (0), counting from the bottom
        blank_row_from_bottom = PUZZLE_DIM - (state.index(0) // PUZZLE_DIM)
        if blank_row_from_bottom % 2 == 0:  # Blank on an even row from bottom
            return inversion_count % 2 == 1
        else:  # Blank on an odd row from bottom
            return inversion_count % 2 == 0

In [ ]:
def get_neighbors(state):
    """
    Generate all possible moves from the current state.

    Args:
        state (tuple): The current state of the puzzle.

    Returns:
        list: A list of tuples containing the new state and the move leading to it.
    """
    zero_idx = state.index(0)  # Locate the blank tile (0)
    neighbors = []
    zero_row, zero_col = divmod(zero_idx, PUZZLE_DIM)  # Calculate row and column of the blank tile

    for dr, dc, move in MOVES:
        # Calculate the new position for the blank tile
        new_row, new_col = zero_row + dr, zero_col + dc
        if 0 <= new_row < PUZZLE_DIM and 0 <= new_col < PUZZLE_DIM:  # Check if move is valid
            new_idx = new_row * PUZZLE_DIM + new_col
            new_state = list(state)  # Convert tuple to list for modification
            # Swap blank tile with target tile
            new_state[zero_idx], new_state[new_idx] = new_state[new_idx], new_state[zero_idx]
            neighbors.append((tuple(new_state), move))  # Add new state and move to neighbors
    return neighbors

# A* Search

In [ ]:
import heapq

def a_star_solve_puzzle(start, goal):
    """
    Solves the n^2-1 puzzle using A* and returns the sequence of moves.

    Args:
        start (tuple): The initial state of the puzzle.
        goal (tuple): The goal state of the puzzle.

    Returns:
        list: A list of moves (e.g., ["U", "L"]) to solve the puzzle.
    """

    def manhattan_distance(state):
        """Calculates the Manhattan distance heuristic."""
        distance = 0
        for i, tile in enumerate(state):
            if tile == 0:  # Skip the empty tile
                continue
            goal_row, goal_col = divmod(tile - 1, n)
            current_row, current_col = divmod(i, n)
            distance += abs(goal_row - current_row) + abs(goal_col - current_col)
        return distance

    # Priority queue: each entry is (f_score, g_score, current_state, moves_so_far)
    pq = []
    heapq.heappush(pq, (manhattan_distance(start), 0, start, []))
    visited = set()

    while pq:
        f_score, g_score, current_state, moves = heapq.heappop(pq)

        # Check if we've reached the goal
        if current_state == goal:
            return moves

        # Skip already visited states
        if current_state in visited:
            continue
        visited.add(current_state)

        # Explore neighbors
        for neighbor_state, move in get_neighbors(current_state):
            if neighbor_state not in visited:
                # g_score + 1 is the new path cost; add heuristic for f_score
                new_g_score = g_score + 1
                new_f_score = new_g_score + manhattan_distance(neighbor_state)
                heapq.heappush(pq, (new_f_score, new_g_score, neighbor_state, moves + [move]))

    return None  # No solution

In [ ]:
state = generate_initial_state()

goal_state = tuple(range(1, PUZZLE_DIM * PUZZLE_DIM)) + (0,)

print("Initial state:")
print(state)
print()

path = a_star_solve_puzzle(state, goal_state)

print("Path to goal:")
print(path)
print()